In [ ]:
BURNIN = 1000  # Burnin period
N = 10000      # number of simulations
setwd('/Users/bayr.yakashev/Учеба/Levy Processes/MCMC Sim/R_Code')
# input daily returns. "scan" reads each line of the file, and assigns these values to the vector Y.
#Y = scan('DailyData2.txt') 


spx_index_values = read.table('sp2010.txt', header = TRUE, sep = ',')
par(mfrow=c(2,1))
plot(as.Date(as.character(spx_index_values$Date), "%m/%d/%Y"), spx_index_values$SPX.Index, type='l', 
     main='SPX index levels, from 1954 to 2015',
     xlab='year', ylab='SPX index')

spx_index_values$Return = rep(0, dim(spx_index_values)[1])
spx_index_values$Return[2:length(spx_index_values$Return)] = 
  log(spx_index_values$SPX.Index[2:length(spx_index_values$SPX.Index)] / 
        spx_index_values$SPX.Index[1:(length(spx_index_values$SPX.Index)-1)])
data.length = length(spx_index_values$Return)


plot(as.Date(as.character(spx_index_values$Date), "%m/%d/%Y"), spx_index_values$Return, type='l', 
     main='SPX index returns, from 1954 to 2015',
     xlab='year', ylab='SPX returns')

Y = spx_index_values$Return

T = length(Y) - 2

#This would reproduce 'NA' BURNIN+N=2000 times. Here 'NA' means 'not available'. This puts in random values for the first 2000 elements.
mu = rep(NA, BURNIN+N)  
psi = rep(NA, BURNIN+N)
omega = rep(NA, BURNIN+N)
kappa = rep(NA, BURNIN+N)
theta = rep(NA, BURNIN+N)
lambda = rep(NA, BURNIN+N)
mus = rep(NA, BURNIN+N)
sigma2s = rep(NA, BURNIN+N)


#Creates a column matrix ((BURNIN+N) BY 1) of epsilon values in the previous line.
epsilons = rep(NA,(BURNIN+N)*T)
epsilons = matrix(epsilons, BURNIN+N) 
epsilonv = rep(NA,(BURNIN+N)*T)
epsilonv = matrix(epsilonv, BURNIN+N)

# parameter initializations
V = rep(NA,(BURNIN+N)*(T+2))		
V = matrix(V, BURNIN+N)
Z = rep(NA,(BURNIN+N)*(T+2))		
Z = matrix(Z, BURNIN+N)
B = rep(NA,(BURNIN+N)*(T+2))		
B = matrix(B, BURNIN+N)
P = rep(NA,(BURNIN+N)*(T+2))		
P = matrix(P, BURNIN+N)

#Hyperparameters for mu
mustar = rep(NA, BURNIN+N)		
sigstar = rep(NA, BURNIN+N)
mustara = rep(NA, BURNIN+N)
mustarb = rep(NA, BURNIN+N)

#Hyperparameters for psi
psistar = rep(NA, BURNIN+N)			
pstar = rep(NA, BURNIN+N)

#Hyperparameters for omega
alphastar = rep(NA, BURNIN+N)		
betastar = rep(NA, BURNIN+N)
betastara = rep(NA, BURNIN+N)
betastarb = rep(NA, BURNIN+N)

#Hyperparameters for kappa
kappastar = rep(NA, BURNIN+N)		
kappastara = rep(NA, BURNIN+N)
kappastarb = rep(NA, BURNIN+N)
sigmakapstar = rep(NA, BURNIN+N)

#Hyperparameters for theta			
thetastar = rep(NA, BURNIN+N)		
thetastara = rep(NA, BURNIN+N)
thetastarb = rep(NA, BURNIN+N)
sigthstar = rep(NA, BURNIN+N)

#Hyperparameters for lambda
alphalam = rep(NA, BURNIN+N)		
betalam = rep(NA, BURNIN+N)

#Hyperparameters for Z (The jump term)
musstar = rep(NA,(BURNIN+N)*(T+2))	
musstar = matrix(musstar, BURNIN+N)
sigma2sstar = rep(NA,(BURNIN+N)*(T+2))		
sigma2sstar = matrix(sigma2sstar, BURNIN+N)

#Just ended part is for setting random values (NA) to all the variables. 

#Initialize prior hyperparameters
mu0 = 0					
sigma0 = 1
psi0 = 0
p0 = 2
alphatilde0 = 2
betatilde0 = 0.005
kappa0 = 0
sigmakap0 = 1
theta0 = 0
sigmath0 = 1
alphaprime = 2
betaprime = 40
mus0 = 0
S0 = 1
alphas = 5.0
betas = 0.2
deltat=1/252

# Metropolis-Hastings recordings
pi = rep(NA, (BURNIN+N)*(T+2))		
pi = matrix(pi, BURNIN+N)
a = rep(NA,(BURNIN+N)*(T+2))
a = matrix(a, BURNIN+N)
accept = rep(NA, BURNIN+N)

rigamma = function(n, a, b) {
  return(1/rgamma(n, shape = a, rate = b))
}
# Initialize parameters
mu[1] = 0.1
omega[1]=0.02
psi[1] = rnorm(1, psi0, sqrt(omega[1]/(p0)))
#psi[1]=0
#omega[1] = rigamma(1, alphatilde0, betatilde0)
kappa[1]=5
theta[1]=0.15^2
lambda[1] = rbeta(1, alphaprime, betaprime)
mus[1]=0
#mus[1]=rnorm(1,mus0,sqrt(S0))
sigma2s[1]=0.01
#sigma2s[1]=rigamma(1,alphas,betas)
P[1,] = rep(lambda[1],T+2)
B[1,] = rbinom(T+2,1,lambda[1])
Z[1,] = rep(0,T+2)
INITIALV = 0.15^2
SIGMAN = 0.005
V[1,] = rnorm(T+2,INITIALV, SIGMAN)

# Initialize hyperparameters
alphalam[1]=alphaprime                          
betalam[1]=betaprime
musstar[1,] = rep(mus[1],T+2)
sigma2sstar[1,] = rep(sigma2s[1],T+2)

epsilons[1,] = (Y[2:(T+1)] - mu[1]*deltat + 1/2*V[1,1:T]*deltat - Z[1,2:(T+1)]*B[1,2:(T+1)])/sqrt(V[1,1:T]*deltat)
epsilonv[1,] = (V[1,2:(T+1)] - kappa[1]*theta[1]*deltat - (1 - kappa[1]*deltat) * V[1,1:T])/
  sqrt(V[1,1:T]*deltat)

pi[1,] = rep(1,T+2)


for(i in 2:(BURNIN+N)) {		# Beginning of MCMC
  
  # Update mu hyperparameters
  mustara[i] = sum((omega[i-1] + psi[i-1]^2) *(Y[2:(T+1)] + 1/2*V[i-1,1:T]*deltat - Z[i-1,2:(T+1)]*B[i-1,2:(T+1)])/(omega[i-1]*V[i-1,1:T]))
  
  mustarb[i] = sum(psi[i-1]*(V[i-1,2:(T+1)] - kappa[i-1]*theta[i-1]*deltat - 
                               (1 - kappa[i-1]*deltat)*V[i-1,1:T])/(omega[i-1]*V[i-1,1:T]))
  
  mustar[i] = (mustara[i]-mustarb[i]+(mu0/sigma0^2))/
    (deltat*sum((omega[i-1] + psi[i-1]^2)/(omega[i-1]*V[i-1,1:T])) + 1/sigma0^2)
  
  sigstar[i] = sqrt(1/(deltat*sum((omega[i-1] + psi[i-1]^2)/(omega[i-1]*V[i-1,1:T])) + 1/sigma0^2))
  
  mu[i] =  rnorm(1,mustar[i],sigstar[i])	# Update mu
  
  
  
  alphastar[i] = T/2 + alphatilde0  # Update omega and psi hyperparameters
  
  eps = (Y[2:(T+1)] - mu[i]*deltat + 1/2*V[i-1,1:T]*deltat)/sqrt(V[i-1,1:T]*deltat)
  
  epv = (V[i-1,2:(T+1)] - kappa[i-1]*theta[i-1]*deltat - (1 - kappa[i-1]*deltat) * V[i-1,1:T])/
    sqrt(V[i-1,1:T]*deltat)
  
  pstar[i] = p0 + sum(eps^2)
  
  psistar[i] = (p0*psi0 + sum(eps*epv))/pstar[i]
  
  betastara[i] = (1/2)*sum(epv^2)
  
  betastarb[i] = (p0*psi0 + sum(eps*epv))^2/pstar[i]
  
  #betastar[i] = betatilde0 + betastara[i] + (1/2)*p0*psi0^2 - (1/2)*betastarb[i]
  
  betastar[i] = (betatilde0 + betastara[i] + (1/2)*p0*psi0^2 - (1/2)*betastarb[i])
  
  omega[i] = rigamma(1, alphastar[i], betastar[i])	# Update omega
  
  psi[i] = rnorm(1, psistar[i], sqrt(omega[i]/pstar[i]))	# Update psi
  
  # Update theta hyperparameters
  thetastara[i] = sum(kappa[i-1]*(V[i-1,2:(T+1)] - (1 - kappa[i-1]*deltat)*V[i-1,1:T])/
                        (omega[i]*V[i-1,1:T]))
  
  thetastarb[i] = sum(psi[i]*(Y[2:(T+1)] - mu[i]*deltat + (1/2)*V[i-1,1:T]*deltat)*kappa[i-1]/
                        (omega[i]*V[i-1,1:T]))
  
  denom = deltat*sum(kappa[i-1]^2/(omega[i]*V[i-1,1:T])) + 1/sigmath0^2
  
  thetastar[i] = (thetastara[i] - thetastarb[i] + theta0/sigmath0^2)/denom
  
  sigthstar[i] = sqrt(1/denom)
  
  #theta[i] = rnorm(1, thetastar[i], sigthstar[i])	# Update theta
  u = runif(1,0,1)					# Updating theta using truncated normal
  eval = (1 - pnorm(0, thetastar[i], sigthstar[i]))*u + pnorm(0, thetastar[i], sigthstar[i])
  theta[i] = qnorm(eval, thetastar[i], sigthstar[i])
  
  
  # Update kappa hyperparameters
  kappastara[i] = sum((theta[i] - V[i-1,1:T])*(V[i-1,2:(T+1)]-V[i-1,1:T])/
                        (omega[i]*V[i-1,1:T]))
  
  kappastarb[i] = sum(psi[i]*(Y[2:(T+1)] - mu[i]*deltat + (1/2)*V[i-1,1:T]*deltat)*(theta[i] - V[i-1,1:T])/
                        (omega[i]*V[i-1,1:T]))
  
  denom = deltat*sum((V[i-1,1:T]-theta[i])^2/(omega[i]*V[i-1,1:T])) + 1/sigmakap0^2
  
  kappastar[i] = (kappastara[i] - kappastarb[i] + kappa0/sigmakap0^2)/denom
  
  sigmakapstar[i] = sqrt(1/denom)
  
  #kappa[i] = rnorm(1, kappastar[i], sigmakapstar[i])	# Update kappa
  u = runif(1,0,1)					# Updating kappa using truncated normal
  eval = (1 - pnorm(0, kappastar[i], sigmakapstar[i]))*u + pnorm(0, kappastar[i], sigmakapstar[i])
  kappa[i] = qnorm(eval, kappastar[i], sigmakapstar[i])
  
  
  V[i,]=rep(-1,T+2)
  for(j in 1:(T+2)){
    while(V[i,j]<0){
      V[i,j] = V[i-1,j] + rnorm(1, 0, SIGMAN)	}}	# Update V - propose new values !!!!!!!!! Random Walk and Independent [!]
   
  count = T+2
  
  epsnew1 = (Y[2] - mu[i]*deltat + 1/2*V[i,1]*deltat)/sqrt(V[i,1]*deltat)
    
  epsold1 = (Y[2] - mu[i]*deltat + 1/2*V[i-1,1]*deltat)/sqrt(V[i-1,1]*deltat)
  
  epvnew1 = (V[i-1,2] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i,1])/
    sqrt(V[i,1]*deltat)
  
  epvold1 = (V[i-1,2] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i-1,1])/
    sqrt(V[i-1,1]*deltat)
  
  pVnew = (1/(V[i,1]*deltat))*exp(-(1/(2*omega[i]))*((omega[i]+psi[i]^2)*epsnew1^2
                                                     - 2* psi[i]*epsnew1*epvnew1 + epvnew1^2))
  
  pVold = (1/(V[i-1,1]*deltat))*exp(-(1/(2*omega[i]))*((omega[i]+psi[i]^2)*epsold1^2
                                                       - 2* psi[i]*epsold1*epvold1 + epvold1^2))
  
  pi[i,1] = pVnew/pVold
  
  if(V[i,1] <= 0) {
    pi[i,1] = 0
  }
  
  a[i,1] = min(pi[i,1],1)
  
  u = runif(1)
  if(u > a[i,1]) {
    V[i,1] = V[i-1,1]
    count = count - 1
  }
  
  
  for(j in 2:(T+1)){
    
    epsnew1 = (Y[(j+1)] - mu[i]*deltat + 1/2*V[i,j]*deltat)/sqrt(V[i,j]*deltat)
    
    
    epvnew1 = (V[i-1,(j+1)] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i,j])/
      sqrt(V[i,j]*deltat)
    
    epvold1 = (V[i-1,(j+1)] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i-1,j])/
      sqrt(V[i-1,j]*deltat)
    
    epsnew0 = (Y[j] - mu[i]*deltat + 1/2*V[i,(j-1)]*deltat)/sqrt(V[i,(j-1)]*deltat)
    
    epsold0 = (Y[j] - mu[i]*deltat + 1/2*V[i,(j-1)]*deltat)/sqrt(V[i,(j-1)]*deltat)
    
    epvnew0 = (V[i,j] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i,(j-1)])/
      sqrt(V[i,(j-1)]*deltat)
    
    epvold0 = (V[i-1,j] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i,(j-1)])/
      sqrt(V[i,(j-1)]*deltat)
    
    pVnew = (1/(V[i,j]*deltat))*exp(-(1/(2*omega[i]))*((omega[i]+psi[i]^2)*epsnew1^2
                                                       - 2* psi[i]*epsnew1*epvnew1 + epvnew1^2)) *
      exp(-(1/(2*omega[i]))*(-2*psi[i]*epsnew0*epvnew0 + epvnew0^2))
    
    pVold = (1/(V[i-1,j]*deltat))*exp(-(1/(2*omega[i]))*((omega[i]+psi[i]^2)*epsold1^2
                                                         - 2* psi[i]*epsold1*epvold1 + epvold1^2)) *
      exp(-(1/(2*omega[i]))*(-2*psi[i]*epsold0*epvold0 + epvold0^2))
    
    pi[i,j] = pVnew/pVold
    
    if(V[i,j] <= 0) {
      pi[i,j] = 0
    }
    
    a[i,j] = min(pi[i,j],1)
    
    u = runif(1)
    if(u > a[i,j]) {
      V[i,j] = V[i-1,j]
      count = count - 1
    }
  }
  
  epsnew0 = (Y[(T+2)] - mu[i]*deltat + 1/2*V[i,(T+1)]*deltat)/sqrt(V[i,(T+1)]*deltat)
  
  epsold0 = (Y[(T+2)] - mu[i]*deltat + 1/2*V[i,(T+1)]*deltat)/sqrt(V[i,(T+1)]*deltat)
  
  epvnew0 = (V[i,(T+2)] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i,(T+1)])/
    sqrt(V[i,(T+1)]*deltat)
  
  epvold0 = (V[i-1,(T+2)] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i,(T+1)])/
    sqrt(V[i,(T+1)]*deltat)
  
  
  pVnew = exp(-(1/(2*omega[i]))*(-2*psi[i]*epsnew0*epvnew0 + epvnew0^2))
  
  pVold = exp(-(1/(2*omega[i]))*(-2*psi[i]*epsold0*epvold0 + epvold0^2))
  
  
  pi[i,(T+2)] = pVnew/pVold
  
  if(V[i,(T+2)] <= 0) {
    pi[i,(T+2)] = 0
  }
  
  a[i,(T+2)] = min(pi[i,(T+2)],1)
  
  u = runif(1)
  if(u > a[i,(T+2)]) {
    V[i,(T+2)] = V[i-1,(T+2)]
    count = count - 1
  }
  
  accept[i] = count/T
  print(i)
  
  
  
  musstar[i,1] = mus[i-1]
  sigma2sstar[i,1] = sigma2s[i-1]
  Z[i,1] = rnorm(1, musstar[i,1], sqrt(sigma2sstar[i,1]))
  
  for (j in 2:(T+2)){			# update Z hyperparameters
    
    musa = (omega[i] + psi[i]^2) *(Y[j] + 1/2*V[i,j-1]*deltat - mu[i]*deltat)/(omega[i]*V[i,j-1]*deltat)
    
    musb = psi[i]*(V[i,j] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat)*V[i,j-1])/(omega[i]*V[i,j-1]*deltat)
    
    musstar[i,j] = (musa*B[i-1,j] - musb*B[i-1,j] + (mus[i-1]/sigma2s[i-1]))/(((omega[i] + psi[i]^2)/(omega[i]*V[i,j-1]*deltat))*B[i-1,j] + 1/sigma2s[i-1])
    
    
    sigma2sstar[i,j] = 1/(((omega[i] + psi[i]^2)/(omega[i]*V[i,j-1]*deltat))*B[i-1,j] + 1/sigma2s[i-1])
    
    Z[i,j] =  rnorm(1,musstar[i,j],sqrt(sigma2sstar[i,j]))	# Update Z
  }
  
  
  P[i,1] = lambda[i-1]
  B[i,1] = rbinom(1, 1, P[i,1])
  
  for(j in 2:(T+2)){			# update B hyperparameters
    A = ((omega[i] + psi[i]^2)*(Z[i,j]^2 - 2*Z[i,j]*(Y[j] - mu[i]*deltat + (1/2)*V[i,j-1]*deltat))
         + 2 *psi[i]*(V[i,j] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat)*V[i,j-1])*Z[i,j])/ (omega[i]*V[i,j-1]*deltat)
    
    P[i,j] = 1/((((1 - lambda[i-1]) *exp((1/2)*A))/(lambda[i-1]))+1)
    
    B[i,j] = rbinom(1, 1, P[i,j])		# update B
  }
  
  alphalam[i] = alphaprime + sum(B[i,2:(T+1)])	# update lambda hyperparameters
  betalam[i] = betaprime + T - sum(B[i,2:(T+1)])
  lambda[i] = rbeta(1, alphalam[i], betalam[i])	# update lambda
  
  musstar2 = sum(Z[i,2:(T+1)]*B[i,2:(T+1)]/sigma2s[i-1])/(1/S0+sum(B[i,2:(T+1)]/sigma2s[i-1]))
  sigma2sstar2 = 1/(1/S0+sum(B[i,2:(T+1)]/sigma2s[i-1]))
  
  mus[i] = rnorm(1, musstar2, sqrt(sigma2sstar2))	# update mus
  
  alphastar2 = alphas + sum(B[i,2:(T+1)])/2
  betastar2 = betas + (1/2)*sum((Z[i,2:(T+1)]-mus[i])^2*B[i,2:(T+1)])
  
  sigma2s[i] = rigamma(1, alphastar2, betastar2)	# update sigma2s
  
  # keep track of epsilons
  
  epsilons[i,] = (Y[2:(T+1)] - mu[i]*deltat + 1/2*V[i,1:T]*deltat - Z[i,2:(T+1)]*B[i,2:(T+1)])/sqrt(V[i,1:T]*deltat)
  epsilonv[i,] = (V[i,2:(T+1)] - kappa[i]*theta[i]*deltat - (1 - kappa[i]*deltat) * V[i,1:T])/
    sqrt(V[i,1:T]*deltat)
  
  
}

rho=psi/sqrt(psi^2+omega)
sigma2v=(psi^2+omega)


# Analysis  ------------------
# Bates
mu_bates = mu
theta_bates = theta
kappa_bates = kappa
mus_bates = mus
sigma2s_bates = sigma2s 
Z_bates = Z
str(Z_bates)
lambda_bates = lambda
sigma2v_bates = sigma2v
rho_bates = rho
V_bates = V
str(V)
epsilons_bates = epsilons
epsilonv_bates = epsilonv
plot(epsilons_bates[11000,], type = 'l')
B_bates = B
str(B_bates)
#-------Перезапись переменных 




spx_index_values = read.table('sp2010.txt', header = TRUE, sep = ',')
par(mfrow=c(2,1))
plot(as.Date(as.character(spx_index_values$Date), "%m/%d/%Y"), spx_index_values$SPX.Index, type='l', 
     main='SPX Индекс, с 2008 по 2010', col="blue",
     xlab='year', ylab='SPX index')

# Q-Q график для доходности 
#par(mfrow=c(1,1))
plot(as.Date(as.character(spx_index_values$Date), "%m/%d/%Y"), Y, type='l', 
     main='SPX Доходность, с 2008 по 2010', col="blue",
     xlab='год', ylab='SPX доходность')

dens <- density(Y, bw = "ucv", n = length((Y)))
plot(dens$x, dens$y, type = "l", lwd = 1, col="red", main = 'Empirical density return vs Normal')
lines(dens$x, dnorm(dens$x, mean = mean(Y), sd = sd(Y)),
      lty = "dashed", lwd = 2)

#-----Гипотеза о логнормальности доходности
par(mfrow=c(1,1))
dens <- density(Y, bw = "ucv", n = length((Y)))
plot(dens$x, dens$y, type = "l", lwd = 1, col="red")
lines(dens$x, dnorm(dens$x, mean = mean(Y), sd = sd(Y)),
      lty = "dashed", lwd = 2)

ks.test(unique(Y), "pnorm", mean = mean(Y), sd = sd(Y))
shapiro.test(Y)

qqplot(rnorm(n=10^3,mean=mean(Y),sd=sd(Y)),Y,pch=16)
abline(0,1,lwd=2)

#--------------------------------
#-----Гипотиза о нормальности остатков
par(mfrow=c(1,1))
dens <- density(epsilonv[11000,], bw = "ucv", n = length((epsilonv[11000,])))
plot(dens$x, dens$y, type = "l", lwd = 1, col="red")
lines(dens$x, dnorm(dens$x, mean = mean(epsilonv[11000,]), sd = sd(epsilonv[11000,])),
      lty = "dashed", lwd = 2)

ks.test(unique(epsilonv[11000,]), "pnorm", mean = mean(epsilonv[11000,]), sd = sd(epsilonv[11000,]))
shapiro.test(epsilonv[11000,])

qqplot(rnorm(n=10^3,mean=mean(epsilons[11000,]),sd=sd(epsilons[11000,])),epsilons[11000,],pch=16)
abline(0,1,lwd=2)

#--------------------------------
par(mfrow=c(1,1))
dens <- density(epsilonv_bates[11000,], bw = "ucv", n = length((epsilonv_bates[11000,])))
plot(dens$x, dens$y, type = "l", lwd = 1, col="red")
lines(dens$x, dnorm(dens$x, mean = mean(epsilonv_bates[11000,]), sd = sd(epsilonv_bates[11000,])),
      lty = "dashed", lwd = 2)

ks.test(unique(epsilonv_bates[11000,]), "pnorm", mean = mean(epsilonv_bates[11000,]), sd = sd(epsilonv_bates[11000,]))
shapiro.test(epsilonv_bates[11000,])

qqplot(rnorm(n=10^3,mean=mean(epsilonv_bates[11000,]),sd=sd(epsilonv_bates[11000,])),epsilonv_bates[11000,],pch=16)
abline(0,1,lwd=2)

#--------------------------------



str(Y)
dens <- density(mu, bw = "ucv", n = length((mu)))
plot(dens$x, dens$y, type = "l", lwd = 1, col="red")
lines(dens$x, dnorm(dens$x, mean = mean(mu), sd = sd(mu)),
      lty = "dashed", lwd = 2)

#----------------------------------
par(mfrow=c(2,1))
str(epsilons)
dens <- density(epsilonv[11000,], bw = "ucv", n = length((epsilonv[11000,])))
plot(dens$x, dens$y, type = "l", lwd = 1, col="red")
lines(dens$x, dnorm(dens$x, mean = mean(epsilonv[11000,]), sd = sd(epsilonv[11000,])),
      lty = "dashed", lwd = 2)
dens <- density(epsilonv_bates[11000,], bw = "ucv", n = length((epsilonv_bates[11000,])))
plot(dens$x, dens$y, type = "l", lwd = 1, col="red")
lines(dens$x, dnorm(dens$x, mean = mean(epsilonv_bates[11000,]), sd = sd(epsilonv_bates[11000,])),
      lty = "dashed", lwd = 2)



shapiro.test(epsilonv[11000,])
shapiro.test(epsilonv_bates[11000,])


qqplot(rnorm(n=10^3,mean=mean(epsilonv[11000,]),sd=sd(epsilonv[11000,])),epsilonv[11000,],pch=16)
abline(0,1,lwd=2)
qqplot(rnorm(n=10^3,mean=mean(epsilonv_bates[11000,]),sd=sd(epsilonv_bates[11000,])),epsilonv_bates[11000,],pch=16)
abline(0,1,lwd=2)

#----------------------
par(mfrow=c(2,1))
dens_epss <- density(epsilons[100,], bw = "ucv", n = length((epsilons[100,])))
plot(dens_epss$x, dens_epss$y, type = "l", lwd = 1, col="red")
lines(dens_epss$x, dnorm(dens_epss$x, mean = mean(epsilons[100,]), sd = sd(epsilons[100,])),
      lty = "dashed", lwd = 2)

dens_epss <- density(epsilons_bates[10000,], bw = "ucv", n = length((epsilons_bates[10000,])))
plot(dens_epss$x, dens_epss$y, type = "l", lwd = 1, col="red")
lines(dens_epss$x, dnorm(dens_epss$x, mean = mean(epsilons_bates[10000,]), sd = sd(epsilons_bates[10000,])),
      lty = "dashed", lwd = 2)

shapiro.test(epsilons[100,])
shapiro.test(epsilons_bates[100,])


par(mfrow=c(2,1))
qqplot(rnorm(n=10^3,mean=mean(epsilons[100,]),sd=sd(epsilons[100,])),epsilons[100,],pch=16)
abline(0,1,lwd=2)
qqplot(rnorm(n=10^3,mean=mean(epsilons_bates[10000,]),sd=sd(epsilons_bates[10000,])),epsilons_bates[10000,],pch=16)
abline(0,1,lwd=2)


#---------------------Volatility-----------
par(mfrow=c(2,1))
plot(V[100,], type = "l", lwd = 1, col="red")
lines(V_bates[100,], type = "l", lwd = 1, col="blue")

plot(Y, type = "l", lwd = 1, col="red")


#---------------------Jump-----------------
par(mfrow=c(1,1))
plot(Z_bates[100,], type = "l", lwd = 1, col="red")
mean(Z_bates[11000,])

acf(Z_bates[3001:length(Z_bates)])

#---------------------Jump-----------------
par(mfrow=c(2,1))

plot(Y, type = "l", lwd = 1, col="red")

str(lambda_bates)
mean(lambda_bates[3000:length(lambda_bates)])
plot(lambda_bates, type = "l", lwd = 1, col="blue")
str(lambda_bates)

str(B_bates)

plot(B_bates[100,], type = "l", lwd = 1, col="blue")

mean(lambda[3000:10000])




#--------Compare Table 

mean(mu[3001:length(mu)])
sd(mu[3001:length(mu)])
mean(mu_bates[3001:length(mu_bates)])
sd(mu_bates[3001:length(mu_bates)])


mean(theta[3001:length(theta)])
sd(theta[3001:length(theta)])
mean(theta_bates[3001:length(theta_bates)])
sd(theta_bates[3001:length(theta_bates)])


mean(kappa[3001:length(kappa)])
sd(kappa[3001:length(kappa)])
mean(kappa_bates[3001:length(kappa_bates)])
sd(kappa_bates[3001:length(kappa_bates)])


mean(rho[3001:length(rho)])
sd(rho[3001:length(rho)])
mean(rho_bates[3001:length(rho_bates)])
sd(rho_bates[3001:length(rho_bates)])

acf(mu_bates[3001:length(mu_bates)])

savehistory(file = "Result.Rhistory")
save.image(file = 'Report.RData')
